In [ ]:
import zarr
import xarray as xr
import numpy as np
import time
import math
from geopy.distance import geodesic

In [ ]:
st = time.time()
ds = xr.open_zarr(
    'GEBCO_2022_sub_ice_topo.zarr', chunks='auto', 
    group='gebco', decode_cf=False, decode_times=False)

et = time.time()
print('Exe time: ', et-st, 'sec')
ds

In [ ]:
# base=90 for latitude; base=180 for longitude
def gridded_arcsec(x, base=90, arc=3600/15):
    return ((int(x) + base)* arc + math.ceil((x-int(x))*arc))

arc = int(3600/15) #arc
print(gridded_arcsec(0, 180, arc))
print(gridded_arcsec(0, 90, arc))
print(gridded_arcsec(-90, 90, arc))
print(gridded_arcsec(90, 90, arc))
print(gridded_arcsec(-180, 180, arc))
print(gridded_arcsec(180, 180, arc))
print(gridded_arcsec(0.0015, 180, arc))
print(gridded_arcsec(-0.00207, 180, arc))
print(gridded_arcsec(-85.375, 90, arc))
print(gridded_arcsec(48.37918, 90, arc))


In [58]:
def curDist(loc, dis=np.empty(shape=[0, 1], dtype=float)):
    if len(loc) < 2:
        return None
    lk = len(loc)-1
    return(np.append(dis, 
                     geodesic((loc[lk-1, 1], loc[lk-1, 0]),
                              (loc[lk, 1], loc[lk, 0])).km))

curDist(np.array([[121.53967066033685,25.070035818788032], [121.5649924414543,25.06911681954908]]))

array([2.55682213])

In [65]:
arcsec = 15
arc = int(3600/arcsec) #15 arc-second
basex = 180 #-180 - 180 <==> 0 - 360, half is 180
basey = 90  #-90 - 90 <==> 0 - 180, half is 90
halfxidx = 180 * arc   #in netcdf, longitude length = 86400
halfyidx = 90 * arc    #in netcdf, latitude length = 43200
lon = [122.26981235231769,122.46580467738616,122.52322927209325] 
#test bug (and solved)
#[121.53967066033685,121.5649924414543] 
#[121.52789789212326,121.58410196573966,121.57161216694406,121.53726523073809,121.53664076385839,121.57723257849847] 
#[123, 123.33, 123.38]
lat = [24.53014834691002,24.484704345640385,24.43356023689588]  
#[25.070035818788032,25.06911681954908] 
#[25.054700205734235,25.033770970073878,24.993599440972766,24.982847101743943,24.942092803893317,24.930769789455212] 
#[23.782329893520448,23.769188442795453]
subsetFlag = True
zmode = "line"
format = "row"
if (len(lon) != len(lat)):
    print("Input Error: check longitude, latitude length not equal")
    #return
# May not have exact lon, lat in gridded lon-lat, so need to calculate index  
elif (len(lon) == 1):
    st = time.time()
    lon0 = gridded_arcsec(lon[0], basex, arc)
    lat0 = gridded_arcsec(lat[0], basey, arc)
    mlon0x = ds["lon"][lon0]
    mlat0x = ds["lat"][lat0]
    st1 = ds.sel(lon=mlon0x, lat=mlat0x)
    loc1 = [lon[0], lat[0]]
    xt1 = np.array([st1['elevation'].values])
    et = time.time()
    print('Exe time: ', et-st, 'sec')
    print(xt1)
else:
    st = time.time()
    idx1 = np.empty(shape=[0, 2], dtype=np.int16)
    loc1 = np.empty(shape=[0, 2], dtype=float)
    dis1 = np.array([0.0])
    mlon0 = np.min(lon)
    #mlon1 = np.max(lon)
    mlat0 = np.min(lat)
    #mlat1 = np.max(lat)
    #ds_s1 = ds.sel(lon=slice(mlon0, mlon1), lat=slice(mlat0, mlat1))
    mlonbase = gridded_arcsec(mlon0, basex, arc) if subsetFlag else 0 #if do subsetting dataset, reference-0-x,y should be biased
    mlatbase = gridded_arcsec(mlat0, basey, arc) if subsetFlag else 0
    print(mlonbase, mlatbase)
    #mlonidx1 = gridded_arcsec(mlon1, basex, arc)
    #mlatidx1 = gridded_arcsec(mlat1, basey, arc)
    for i in range(len(lon)-1):
        lonidx0 = gridded_arcsec(lon[i], basex, arc)
        latidx0 = gridded_arcsec(lat[i], basey, arc)
        lonidx1 = gridded_arcsec(lon[i+1], basex, arc)
        latidx1 = gridded_arcsec(lat[i+1], basey, arc)
        #ds_s1 = ds.sel(lon=slice(lon[i], lon[i+1]), lat=slice(lat[i], lat[i+1]))
        #index from gridded_arcsec will change is reference is ds_s1, not ds
        if lonidx0 == lonidx1 and latidx0 == latidx1:
            idx1 = np.append(idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
            loc1 = np.append(loc1, [[lon[i], lat[i]]], axis=0)
            if i >= 1:
                dis1 = np.append(dis1, 0.0, axis=None) #to match the same length
        elif zmode == 'point':
            idx1 = np.append(
                idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
            loc1 = np.append(loc1, [[lon[i], lat[i]]], axis=0)
            dist = geodesic((lat[i], lon[i]),
                            (lat[i+1], lon[i+1])).km
            dis1 = np.append(dis1, dist, axis=None)
            if i == len(lon)-2:
                idx1 = np.append(
                    idx1, [[latidx1-mlatbase, lonidx1-mlonbase]], axis=0)
                loc1 = np.append(loc1, [[lon[i+1], lat[i+1]]], axis=0)            
        else:        
            if lon[i] == lon[i+1]:
                stepi = -1 if latidx0 > latidx1 else 1
                rngi = range(latidx0, latidx1+stepi, stepi)
                leni = len(rngi)
                for k,y in enumerate(rngi):                  
                    locy0 = y/arc - basey
                    locy0i= int(locy0)
                    doty0i= locy0-locy0i-0.25/arc #a small bias to make sure it's in grid
                    locy1 = locy0i + doty0i
                    if (k<(leni-1)) or (stepi == 1 and locy1 < lat[i+1]) or (stepi == -1 and locy1 > lat[i+1]):
                        print("yi: ", y, locy0, locy0i, locy1)
                        idx1 = np.append(idx1, [[y-mlatbase, lonidx0-mlonbase]], axis=0)
                        loc1 = np.append(loc1, [[lon[i], locy1]], axis=0)
                        if i>=1 or k>=1:
                            dis1 = curDist(loc1, dis1)
                            #print("dist: ", dist, " at ", loc1[lk-1, 1], " from ", loc1[lk, 1])                    
            elif lat[i] == lat[i+1]:
                stepi = -1 if lonidx0 > lonidx1 else 1
                rngi = range(lonidx0, lonidx1+stepi, stepi)
                leni = len(rngi)
                for k,x in enumerate(rngi):                    
                    locx0 = x/arc - basex
                    locx0i= int(locx0)
                    dotx0i= locx0-locx0i-0.25/arc #a small bias to make sure it's in grid
                    locx1 = locx0i + dotx0i
                    if (k<(leni-1)) or (stepi == 1 and locx1 < lon[i+1]) or (stepi == -1 and locx1 > lon[i+1]):
                        print("xi: ", x, locx1)
                        idx1 = np.append(idx1, [[latidx0-mlatbase, x-mlonbase]], axis=0)
                        loc1 = np.append(loc1, [[locx1, lat[i]]], axis=0)
                        if i>=1 or k>=1:
                            dis1 = curDist(loc1, dis1)

            else:
                m = (lat[i+1]-lat[i])/(lon[i+1]-lon[i])
                b = lat[i] - m * lon[i] #y = mx + b
                print("m, b:", m ,b)
                if np.absolute(m) <= 1:
                    lidx0 = lonidx0
                    lidx1 = lonidx1
                else: 
                    lidx0 = latidx0
                    lidx1 = latidx1

                stepi = -1 if lidx0 > lidx1 else 1
                rngi = range(lidx0, lidx1+stepi, stepi)
                leni = len(rngi)
                for k,s in enumerate(rngi):
                    if k==0: #should consider internal node not repeated twice
                        idx1 = np.append(idx1, [[latidx0-mlatbase, lonidx0-mlonbase]], axis=0)
                        loc1 = np.append(loc1, [[lon[i], lat[i]]], axis=0)
                        if i >= 1:
                            dis1 = curDist(loc1, dis1)

                    else:
                        if np.absolute(m) <= 1:
                            locx0 = s/arc - basex
                            locx0i= int(locx0)
                            dotx0i= locx0-locx0i-0.25/arc #a small bias to make sure it's in grid
                            locx1 = locx0i + dotx0i 
                            locy1 = m * locx1 + b 
                            if (k<(leni-1)) or (stepi == 1 and locx1 < lon[i+1]) or (stepi == -1 and locx1 > lon[i+1]):
                                y = gridded_arcsec(locy1, basey, arc)
                                idx1 = np.append(idx1, [[y-mlatbase, s-mlonbase]], axis=0)
                                print("line: ", s, locx0, dotx0i, locx1, locy1, y)
                                loc1 = np.append(loc1, [[locx1, locy1]], axis=0)
                                dis1 = curDist(loc1, dis1)

                        else:
                            locy0 = s/arc - basey
                            locy0i= int(locy0)
                            doty0i= locy0-locy0i-0.25/arc
                            locy1 = locy0i + doty0i 
                            locx1 = (locy1 - b)/m 
                            if (k<(leni-1)) or (stepi == 1 and locy1 < lat[i+1]) or (stepi == -1 and locy1 > lat[i+1]):
                                x = gridded_arcsec(locx1, basex, arc)
                                idx1 = np.append(idx1, [[s-mlatbase, x-mlonbase]], axis=0)
                                print("line: ", s, locy0, doty0i, locy1, locx1, x)
                                loc1 = np.append(loc1, [[locx1, locy1]], axis=0)
                                dis1 = curDist(loc1, dis1)
                        #print("ith-turn: ", i, k, len(idx1), len(loc1), len(dis1))

            if i == len(lon)-2: #and lonidx1 != idx1[len(idx1)-1, 1] and latidx1 != idx1[len(idx1)-1, 0]:
                idx1 = np.append(
                    idx1, [[latidx1-mlatbase, lonidx1-mlonbase]], axis=0)
                loc1 = np.append(loc1, [[lon[i+1], lat[i+1]]], axis=0)
                dis1 = curDist(loc1, dis1)
                    
    et = time.time()
    print('Exe time: ', et-st, 'sec')
    print(idx1)
    print(loc1)
    print(dis1)
    np.savetxt("simu/test_loc.csv", loc1, delimiter=",", fmt='%f') 
    np.savetxt("simu/test_dis.csv", dis1, delimiter=",", fmt='%f') 


72545 27465
m, b: -0.23186622870951648 52.88038862206218
line:  72546 122.27499999999998 0.2739583333333106 122.27395833333331 24.529187033927627 27488
line:  72547 122.27916666666664 0.2781249999999735 122.27812499999997 24.52822092464134 27487
line:  72548 122.28333333333336 0.2822916666666932 122.2822916666667 24.527254815355036 27487
line:  72549 122.28750000000002 0.2864583333333561 122.28645833333336 24.526288706068748 27487
line:  72550 122.29166666666669 0.29062500000001895 122.29062500000002 24.52532259678246 27487
line:  72551 122.29583333333335 0.29479166666668183 122.29479166666668 24.52435648749617 27486
line:  72552 122.30000000000001 0.2989583333333447 122.29895833333335 24.523390378209882 27486
line:  72553 122.30416666666667 0.3031250000000076 122.30312500000001 24.522424268923594 27486
line:  72554 122.30833333333334 0.30729166666667046 122.30729166666667 24.521458159637305 27486
line:  72555 122.3125 0.31145833333333334 122.31145833333333 24.520492050351017 27485
lin

In [ ]:
st = time.time()
#mlon0 = np.min(lon) #may cause slice offset to mlonbase, an offset +-1
mlon1 = np.max(lon) + 1.5/arc #to make it larger
#mlat0 = np.min(lat) #may cause slice offset to mlatbase, an offset +-1
mlat1 = np.max(lat) + 1.5/arc
mlon0x= ds["lon"][mlonbase]
mlat0x= ds["lat"][mlatbase]
ds_s1 = ds.sel(lon=slice(mlon0x, mlon1), lat=slice(mlat0x, mlat1)) if subsetFlag else ds
print(ds_s1['elevation'].shape)
#mlonidx0 = gridded_arcsec(mlon0, basex, arc)
#mlatidx0 = gridded_arcsec(mlat0, basey, arc)
#mlonidx1 = gridded_arcsec(mlon1, basex, arc)
#mlatidx1 = gridded_arcsec(mlat1, basey, arc)
loct = tuple(idx1.T)
print(loct)
pt1=ds_s1['elevation'].values[loct]
et = time.time()
print('Get index of points, time: ', et-st, 'sec')
print(pt1)
np.savetxt("simu/test_zseg.csv", pt1, delimiter=",", fmt='%f')

In [ ]:
ds_s1 = ds.sel(lon=slice(mlon0, mlon1), lat=slice(mlat0, mlat1)) 
print(ds_s1)
print(ds["lat"][26720]) #actually more near 21.34, not 21.33 so slice 21.33 makes the offset 1 possibly!!
print(ds["lon"][72720]) #should use print(ds["lat"][26720]) to calcuate back
print(ds["elevation"].values[26720,72720]) 

In [ ]:
loc1[:,1]

In [ ]:
from fastapi.encoders import jsonable_encoder
#from fastapi.responses import JSONResponse
json_data = jsonable_encoder({"longitude": loc1[:,0].tolist(), "latitude": loc1[:,1].tolist(),"z": pt1.tolist()})
#print(json_data)

import pandas as pd

df1 = pd.DataFrame({"longitude": loc1[:,0].tolist(), "latitude": loc1[:,1].tolist(),"z": pt1.tolist()}, columns=['longitude', 'latitude', 'z'])
print(df1)

In [ ]:
from fastapi.encoders import jsonable_encoder
#just some  simple testing for query string
lax = "25.5, 30.22,20"
lox = " 125.33"
#la1 = lax.split(",")
if (',' in lax):
    try:
        la1 = [float(x.strip()) for x in lax.split(',')]
    except ValueError:
        print("Check your input format") 
else: 
    la1 = [float(lax.strip())]

if (',' in lox):
    lo1 = lox.split(",")
else: 
    lo1 = [float(lox.strip())]
print(la1)
print(lo1)

lon0 = gridded_arcsec(123, basex, arc)
lat0 = gridded_arcsec(21.33, basey, arc)
mlon0x = ds["lon"][lon0]
mlat0x = ds["lat"][lat0]
st1 = ds.sel(lon=mlon0x, lat=mlat0x)
xt1 = np.array([st1['elevation'].values])
print(xt1)
jt1 = {"data": xt1.tolist()}
jsonable_encoder(jt1)

In [ ]:
dis1 = np.array([0])
st = time.time()
for i in range(len(loc1)-1):
  dist = geodesic((loc1[i,1],loc1[i,0]), (loc1[i+1,1],loc1[i+1,0])).km
  dis1 = np.append(dis1, dist, axis=None)
et = time.time()
print('Calculate loop of geo dist time: ', et-st, 'sec')
print(dis1)

In [ ]:
#https://stackoverflow.com/questions/72620476/faster-way-to-calculate-distance-from-coordinates-on-dataframe
st = time.time()
df2 = pd.merge(df1.rename(columns={'latitude':'clat','longitude':'clon'}), df1, how='cross') \
            .assign(distance=lambda r: r.apply(lambda x: geodesic((x['clat'],x['clon']),(x['latitude'],x['longitude'])).km, axis=1)
            ) #.drop(columns=['clat','clon'])
et = time.time()
print('Calculate df of geo dist time: ', et-st, 'sec')
print(df2)

In [45]:
# want to solve a situation when cross 180-line or zero-line (prime meridian)
l1a = [-179.95, 25]
l2a = [179,95, 24]
l1b = [-0.05, 25]
l2b = [0.05, 24]

print(geodesic((25,-179.95), (25,179.95)).km)
print(geodesic((25,-0.05), (25,0.05)).km)

def whichSide(l1a, l2a, direction='auto'):
   #posFlag = 'near-zero' #near prime meridian (near-zero) #otherwise, far-zero (near 180-degree)
   if l1a[0] != l2a[0] and np.sign(l1a[0]) != np.sign(l2a[0]):
      lonat1 = 180 + l1a[0]
      lonat2 = 180 + l2a[0]
      dift1 = np.absolute(lonat2 - lonat1)

      lonas1 = 180 - l1a[0] if l1a[0]>=0 else 180 - l2a[0]
      lonas2 = -180- l2a[0] if l2a[0]<0 else -180 - l1a[0]
      dift2 = np.absolute(lonas2 - lonas1) 

      if dift1 < dift2:
         posFlag = 'near-zero'
         print("use forward side", posFlag)
      else:
         posFlag = 'far-zero'
         print("use other side", posFlag)    

whichSide(l1a, l2a)
whichSide(l1b, l2b)
whichSide([-35.95, 25], [35, 24])
whichSide([-130.95, 25], [35, 24])
whichSide([-160.95, 25], [35, 24])




10.095008799533332
10.095008799531035
use other side far-zero
use forward side near-zero
use forward side near-zero
use forward side near-zero
use other side far-zero
